In [ ]:
%websocketconnect 192.168.1.130 --password horsetoe -kbi

In [ ]:
2+2

In [ ]:
global wdt_counter
wdt_counter = 0

# Temp, barometric pressure, and humidity

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

Hooked up to I2C (SDA and SCL)

SDA = machine pin 4 = D2

SCL = machine pin 5 = D1

In [ ]:
import machine

In [ ]:
i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))

In [ ]:
import utime

Sample code at  https://github.com/chrisbalmer/micropython-si7021/blob/master/si7021.py

Data sheet at https://cdn.sparkfun.com/datasheets/Sensors/Weather/Si7021.pdf

`While the measurement is in progress, the option of either clock stretching (Hold Master Mode) or Not Acknowledging read requests (No Hold Master Mode) is available to indicate to the master that the measurement is in progress; the chosen command code determines which mode is used.`

#### Si7021 is at address 0x40 / 64

Measure Relative Humidity, Hold Master Mode 0xE5

Measure Relative Humidity, No Hold Master Mode 0xF5

Measure Temperature, Hold Master Mode 0xE3

Measure Temperature, No Hold Master Mode 0xF3

Read Temperature Value from Previous RH Measurement 0xE0

Reset 0xFE

Write RH/T User Register 1 0xE6

Read RH/T User Register 1 0xE7

Read Electronic ID 1st Byte 0xFA 0x0F

Read Electronic ID 2nd Byte 0xFC 0xC9

Read Firmware Revision 0x84 0xB8

In [ ]:
SI7021_i2c_address = 0x40
SI7021_measure_temperature = bytearray([0xF3])
SI7021_measure_relative_humidity = bytearray([0xF5])
SI7021_read_ID_1 = bytearray([0xFA, 0x0F])
SI7021_read_ID_2 = bytearray([0xFC, 0xC9])

In [ ]:
def convert_bytes_to_integer(bytes_to_convert):
    'Use bitwise operators to convert the bytes into integers.'
    integer = None
    for chunk in bytes_to_convert:
        if not integer:
            integer = chunk
        else:
            integer = integer << 8
            integer = integer | chunk
    return integer

In [ ]:
def get_Si7021_device_info(i2c_object, SI7021_i2c_address):
    '''Get the serial number and the sensor identifier. The identifier is
    part of the bytes returned for the serial number.
    '''
    i2c = i2c_object
    # Serial 1st half
    i2c.writeto(SI7021_i2c_address, SI7021_read_ID_1)
    id1 = bytearray(8)
    utime.sleep(0.025)
    i2c.readfrom_into(SI7021_i2c_address, id1)

    # Serial 2nd half
    i2c.writeto(SI7021_i2c_address, SI7021_read_ID_2)
    id2 = bytearray(6)
    utime.sleep(0.025)
    i2c.readfrom_into(SI7021_i2c_address, id2)

    combined_id = bytearray([id1[0], id1[2], id1[4], id1[6],
                             id2[0], id2[1], id2[3], id2[4]])

    serial = convert_bytes_to_integer(combined_id)
    identifier = get_Si7021_device_identifier(id2[0])

    return serial, identifier


In [ ]:
def get_Si7021_device_identifier(identifier_byte):
    '''Convert the identifier byte to a device identifier. Values are based
    on the information from page 24 of the datasheet.
    '''
    if identifier_byte == 0x00 or identifier_byte == 0xFF:
        return 'engineering sample'
    elif identifier_byte == 0x0D:
        return 'Si7013'
    elif identifier_byte == 0x14:
        return 'Si7020'
    elif identifier_byte == 0x15:
        return 'Si7021'
    else:
        return 'unknown'

In [ ]:
get_Si7021_device_info(i2c, 0x40)

### Get temperature

In [ ]:
temperature_data = bytearray(3)

In [ ]:
i2c.writeto(SI7021_i2c_address, SI7021_measure_temperature)

In [ ]:
i2c.readfrom_into(SI7021_i2c_address, temperature_data)

In [ ]:
print(temperature_data)

In [ ]:
def verify_SI7021_checksum(data):
    crc = 0
    values = data[:2]
    checksum = int(data[-1])
    for value in values:
        crc = crc ^ value
        for _ in range(8, 0, -1):
            if crc & 0x80: #10000000
                crc <<= 1
                crc ^= 0x131 #100110001
            else:
                crc <<= 1
    if crc != checksum:
        return False
    else:
        return True

In [ ]:
verify_SI7021_checksum(temperature_data)

The third byte is a checksum, so only look at the first two for the temperature

In [ ]:
temperature_data_int = convert_bytes_to_integer(temperature_data[:2])

In [ ]:
print(temperature_data_int)

Temperature (C) =  ((175.72\*Temp_Code) / 65536) - 46.85

In [ ]:
((175.72*temperature_data_int) / 65536) - 46.85

In [ ]:
def get_SI7021_temperature_celsius(i2c_object, SI7021_i2c_address):
    'Retrieve data from the sensor and verify checksum'
    i2c = i2c_object
    data = bytearray(3)
    #Measure temperature command = 0xF3
    i2c.writeto(SI7021_i2c_address, SI7021_measure_temperature)
    utime.sleep(0.02)

    i2c.readfrom_into(SI7021_i2c_address, data)
    value = convert_bytes_to_integer(data[:2])
    value = ((175.72*value) / 65536) - 46.85
    verified = verify_SI7021_checksum(data)
    if verified:
        return value
    return False

In [ ]:
get_SI7021_temperature_celsius(i2c, 0x40)

%RH = (125\*RH_Code) / 65536 

In [ ]:
def get_SI7021_relative_humidity(i2c_object, SI7021_i2c_address=0x40):
    i2c = i2c_object
    SI7021_measure_relative_humidity = bytearray([0xF5])
    humidity_data = bytearray(3)
    i2c.writeto(SI7021_i2c_address, SI7021_measure_relative_humidity)
    utime.sleep(0.02)
    i2c.readfrom_into(SI7021_i2c_address, humidity_data)
    value = convert_bytes_to_integer(humidity_data[:2])
    value = ((125*value) / 65536)
    verified = verify_SI7021_checksum(humidity_data)
    if not verified:
        return False
    return value

In [ ]:
get_SI7021_relative_humidity(i2c, 0x40)

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

In [ ]:
import machine
import utime

def convert_bytes_to_integer(bytes_to_convert):
    integer = None
    for chunk in bytes_to_convert:
        if not integer:
            integer = chunk
        else:
            integer = integer << 8
            integer = integer | chunk
    return integer

def verify_SI7021_checksum(data_to_verify):
    crc = 0
    values = data_to_verify[:2]
    checksum = int(data_to_verify[-1])
    for value in values:
        crc = crc ^ value
        for _ in range(8, 0, -1):
            if crc & 0x80: #10000000
                crc <<= 1
                crc ^= 0x131 #100110001
            else:
                crc <<= 1
    if crc == checksum:
        return True
    else:
        return False

def get_SI7021_temperature_celsius(i2c_object, SI7021_i2c_address=0x40):
    SI7021_measure_temperature_command = bytearray([0xF3])
    temperature_data = bytearray(3)
    i2c_object.writeto(SI7021_i2c_address, SI7021_measure_temperature_command)
    utime.sleep(0.01)
    i2c_object.readfrom_into(SI7021_i2c_address, temperature_data)
    value = convert_bytes_to_integer(temperature_data[:2])
    value = ((175.72*value) / 65536) - 46.85
    verified = verify_SI7021_checksum(temperature_data)
    if verified:
        return value
    return False

def get_SI7021_relative_humidity(i2c_object, SI7021_i2c_address=0x40):
    SI7021_measure_relative_humidity = bytearray([0xF5])
    humidity_data = bytearray(3)
    i2c_object.writeto(SI7021_i2c_address, SI7021_measure_relative_humidity)
    utime.sleep(0.02)
    i2c_object.readfrom_into(SI7021_i2c_address, humidity_data)
    value = convert_bytes_to_integer(humidity_data[:2])
    value = ((125*value) / 65536)
    verified = verify_SI7021_checksum(humidity_data)
    if verified:
        return value
    return False

def read_temp_humidity_data(i2c_object, SI7021_i2c_address=0x40):
    return {"temperature_celcius" : get_SI7021_temperature_celsius(i2c_object, SI7021_i2c_address),
            "relative_humidity" : get_SI7021_relative_humidity(i2c_object, SI7021_i2c_address)}

In [ ]:
read_temp_humidity_data(i2c, 0x40)

In [ ]:
import gc
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

In [ ]:
temp_humidity_script = """import machine
import utime

def convert_bytes_to_integer(bytes_to_convert):
    integer = None
    for chunk in bytes_to_convert:
        if not integer:
            integer = chunk
        else:
            integer = integer << 8
            integer = integer | chunk
    return integer

def verify_SI7021_checksum(data_to_verify):
    crc = 0
    values = data_to_verify[:2]
    checksum = int(data_to_verify[-1])
    for value in values:
        crc = crc ^ value
        for _ in range(8, 0, -1):
            if crc & 0x80: #10000000
                crc <<= 1
                crc ^= 0x131 #100110001
            else:
                crc <<= 1
    if crc == checksum:
        return True
    else:
        return False

def get_SI7021_temperature_celsius(i2c_object, SI7021_i2c_address=0x40):
    SI7021_measure_temperature_command = bytearray([0xF3])
    temperature_data = bytearray(3)
    i2c_object.writeto(SI7021_i2c_address, SI7021_measure_temperature_command)
    utime.sleep(0.01)
    i2c_object.readfrom_into(SI7021_i2c_address, temperature_data)
    value = convert_bytes_to_integer(temperature_data[:2])
    value = ((175.72*value) / 65536) - 46.85
    verified = verify_SI7021_checksum(temperature_data)
    if verified:
        return value
    return False

def get_SI7021_relative_humidity(i2c_object, SI7021_i2c_address=0x40):
    SI7021_measure_relative_humidity = bytearray([0xF5])
    humidity_data = bytearray(3)
    i2c_object.writeto(SI7021_i2c_address, SI7021_measure_relative_humidity)
    utime.sleep(0.02)
    i2c_object.readfrom_into(SI7021_i2c_address, humidity_data)
    value = convert_bytes_to_integer(humidity_data[:2])
    value = ((125*value) / 65536)
    verified = verify_SI7021_checksum(humidity_data)
    if verified:
        return value
    return False

def read_temp_humidity_data(i2c_object, SI7021_i2c_address=0x40):
    return {"temperature_celcius" : get_SI7021_temperature_celsius(i2c_object, SI7021_i2c_address),
            "relative_humidity" : get_SI7021_relative_humidity(i2c_object, SI7021_i2c_address)}
"""

In [ ]:
import os
print(os.listdir())

In [ ]:
with open('temp_humidity_sensor.py') as f:
    print(f.read())

In [ ]:
with open('temp_humidity_sensor.txt', 'w') as f:
    f.write(temp_humidity_script)

### Double check the write

In [ ]:
with open('temp_humidity_sensor.txt') as f:
    print(f.read())

## Move temp file to `boot.py`

In [ ]:
os.rename('temp_humidity_sensor.txt', 'temp_humidity_sensor.py')

In [ ]:
%disconnect

In [ ]:
machine.reset()